In [1]:
import json
tweets = []
for line in open('./data/Gamergate.json', 'r'):
    tweets.append(json.loads(line))
tweets[0]

{'retweet_count': 338,
 'user': {'description': 'Seattle-based amateur chef and gamer with deep passion for theater and music.',
  'followers_count': 582,
  'default_profile_image': False,
  'profile_image_url_https': 'https://pbs.twimg.com/profile_images/977029984966533120/94BsR1Ae_normal.jpg',
  'profile_link_color': 'FF0000',
  'created_at': 'Thu Feb 12 19:27:48 +0000 2009',
  'profile_use_background_image': True,
  'time_zone': 'Pacific Time (US & Canada)',
  'has_extended_profile': True,
  'following': False,
  'translator_type': 'none',
  'profile_sidebar_border_color': '65B0DA',
  'follow_request_sent': False,
  'profile_background_tile': True,
  'id': 20708129,
  'utc_offset': -25200,
  'profile_text_color': '3D1957',
  'location': 'Seattle, WA',
  'lang': 'en',
  'listed_count': 54,
  'friends_count': 414,
  'profile_sidebar_fill_color': '7AC3EE',
  'profile_background_color': '642D8B',
  'protected': False,
  'profile_image_url': 'http://pbs.twimg.com/profile_images/977029984

A. How many tweets are in this dataset? (5 points)

In [2]:
n_tweet = len(tweets)
n_tweet

104914

In [3]:
tweets[0]['user']['id']

20708129

In [4]:
user_id = []
for tweet in tweets:
    user_id.append(tweet['user']['id'])
user_id[0]

20708129

In [5]:
from pyspark import SparkContext
try:
    sc
except NameError:
    sc = SparkContext("local[*]", "PySpark Tutorial")

In [9]:
user_id_rdd = sc.parallelize(user_id)
user_id_rdd.take(5)

[20708129, 251045973, 1091189214, 2847688135, 2492324996]

In [10]:
user_id_tuple_rdd = user_id_rdd.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)
user_id_tuple_rdd.take(5)

[(18746944, 247),
 (2835016336, 104),
 (37632528, 2),
 (2790540392, 50),
 (2181045176, 24)]

B. How many unique users are in this dataset? Hint: each user has its unique user
id. (5 points)

In [11]:
n_user = user_id_tuple_rdd.count()
n_user

21049

In [15]:
user_name_fo = []
for tweet in tweets:
    user_name_fo.append((tweet['user']['screen_name'], tweet['user']['followers_count']))
user_name_fo[0]

('GrrlGotGame', 582)

In [17]:
user_name_fo_rdd = sc.parallelize(user_name_fo).sortBy(lambda x: x[1], False)
user_name_fo_rdd.take(5)

[('washingtonpost', 12584968),
 ('TheOnion', 11427200),
 ('WIRED', 10587728),
 ('WIRED', 10587719),
 ('guardian', 7195741)]

C. Identify top 3 users with most followers. We would need you to output the
screen name of these users along with the number of followers they each have.
(5 points)

In [18]:
popular_users = user_name_fo_rdd.take(3)
popular_users

[('washingtonpost', 12584968), ('TheOnion', 11427200), ('WIRED', 10587728)]

In [24]:
tweets[0]['created_at'].split()[0]

'Tue'

In [26]:
tuesday = []
tue_cnt = 0
for tweet in tweets:
    if tweet['created_at'].split()[0] == 'Tue':
        tue_cnt += 1
tue_cnt

44558

In [41]:
weekday = []
for tweet in tweets:
    weekday.append(tweet['created_at'].split()[0])
weekday_rdd = sc.parallelize(weekday)
weekday_tuple_rdd = weekday_rdd.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)
weekday_cnt = weekday_tuple_rdd.collect()
weekday_cnt

[('Wed', 43454), ('Thu', 16902), ('Tue', 44558)]

In [43]:
weekday_dict = {}
for cnt in weekday_cnt:
    weekday_dict[cnt[0]]  = cnt[1]
weekday_dict

{'Wed': 43454, 'Thu': 16902, 'Tue': 44558}

D. Each tweet has an associated date to when it is created. Identify the number of
tweets that are created on Tuesday. (5 points)
Input

In [44]:
Tuesday_Tweet = weekday_dict['Tue']
Tuesday_Tweet

44558